In [1]:
import pandas as pd

# 1. Préparation des données

### a. Import du dataset et inspection rapide

In [2]:
df = pd.read_csv("../data/lille_2022.csv")

In [3]:
df.head()

,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,prix_m2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,0,1.0,Maison,NaN,82.0,5.0,S,NaN,70.0,2681.707317
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,2,2.0,Appartement,NaN,63.0,3.0,NaN,NaN,NaN,2936.507937
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,2,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN,inf
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN,inf
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN,inf


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11354 entries, 0 to 11353
Data columns (total 44 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Identifiant de document     0 non-null      float64
 1   Reference document          0 non-null      float64
 2   1 Articles CGI              0 non-null      float64
 3   2 Articles CGI              0 non-null      float64
 4   3 Articles CGI              0 non-null      float64
 5   4 Articles CGI              0 non-null      float64
 6   5 Articles CGI              0 non-null      float64
 7   No disposition              11354 non-null  int64  
 8   Date mutation               11354 non-null  object 
 9   Nature mutation             11354 non-null  object 
 10  Valeur fonciere             11354 non-null  float64
 11  No voie                     11248 non-null  float64
 12  B/T/Q                       652 non-null    object 
 13  Type de voie                113

### b. Filtre des biens de 4 pièces uniquement

In [5]:
df = df[df["Nombre pieces principales"] == 4.0]

### c. Création de datasets spécifiques maisons / appartements

In [6]:
df_maisons = df[df["Type local"] == "Maison"]

In [7]:
df_appartements = df[df["Type local"] == "Appartement"]

### d. Sélection des colonnes utiles

In [8]:
colonnes_utiles = ["Surface reelle bati", "Nombre pieces principales", "Type local", "Surface terrain", "Nombre de lots", "Valeur fonciere", "prix_m2"]

df_maisons = df_maisons[colonnes_utiles]
df_appartements = df_appartements[colonnes_utiles]

### e. Vérification des données

In [9]:
df_maisons.head()

,Surface reelle bati,Nombre pieces principales,Type local,Surface terrain,Nombre de lots,Valeur fonciere,prix_m2
18,165.0,4.0,Maison,121.0,0,305000.0,1848.484848
65,64.0,4.0,Maison,127.0,0,226900.0,3545.312500
161,70.0,4.0,Maison,192.0,0,205890.0,2941.285714
168,62.0,4.0,Maison,96.0,0,159500.0,2572.580645
176,72.0,4.0,Maison,135.0,0,259900.0,3609.722222


In [10]:
df_maisons.shape

(354, 7)

In [11]:
df_maisons.describe()

,Surface reelle bati,Nombre pieces principales,Surface terrain,Nombre de lots,Valeur fonciere,prix_m2
count,354.000000,354.0,340.000000,354.000000,354.000000,354.000000
mean,84.310734,4.0,142.638235,0.039548,251205.864379,3003.594635
std,19.462888,0.0,84.800103,0.195171,105865.398518,1116.900260
min,44.000000,4.0,31.000000,0.000000,16000.000000,144.144144
25%,71.000000,4.0,84.750000,0.000000,185032.500000,2291.390977
50%,80.500000,4.0,127.000000,0.000000,237550.000000,2847.948718
75%,95.000000,4.0,175.250000,0.000000,290000.000000,3481.995492
max,165.000000,4.0,607.000000,1.000000,753750.000000,9192.073171


In [12]:
df_appartements.head()

,Surface reelle bati,Nombre pieces principales,Type local,Surface terrain,Nombre de lots,Valeur fonciere,prix_m2
23,92.0,4.0,Appartement,NaN,1,367000.0,3989.130435
26,70.0,4.0,Appartement,NaN,2,170000.0,2428.571429
58,105.0,4.0,Appartement,NaN,2,355300.0,3383.809524
62,120.0,4.0,Appartement,NaN,1,472500.0,3937.500000
113,71.0,4.0,Appartement,NaN,1,229900.0,3238.028169


In [13]:
df_appartements.shape

(435, 7)

In [14]:
df_appartements.describe()

,Surface reelle bati,Nombre pieces principales,Surface terrain,Nombre de lots,Valeur fonciere,prix_m2
count,435.000000,435.0,49.000000,435.000000,4.350000e+02,435.000000
mean,90.450575,4.0,232.714286,1.434483,4.516911e+05,4989.066032
std,23.340679,0.0,193.109533,0.865204,6.368548e+05,7087.899307
min,18.000000,4.0,43.000000,0.000000,1.000000e+00,0.011494
25%,77.000000,4.0,136.000000,1.000000,2.150000e+05,2617.078231
50%,85.000000,4.0,176.000000,1.000000,2.850000e+05,3285.714286
75%,99.000000,4.0,306.000000,2.000000,4.297500e+05,4310.382231
max,215.000000,4.0,1207.000000,8.000000,4.738000e+06,52065.934066


Nous pouvons voir que sur 435 appartements, 49 ont une information Surface terrain (nous pouvons supposer que seuls 49 appartement possédaient un terrain à la vente). Il faudra donc traiter cette information intelligemment :  
- Supprimer la colonne car cela ne représente que 10-11% des appartements, et donc peu pertinente voire aberrante parfois.
- Garder la colonne mais la transformer en variable binaire (possède un terrain ou non)
- Garder la colonne telle quelle avec une valeur a 0 pour ceux ne possèdant pas de terrain

Ici je décide de supprimer la colonne car même si l'information pouvait être précieuse, elle ne représente que peu de cas, et cela va éviter d'introduire du bruit pour le modèle.

### f. Traitement des données

Maison : Remplacement des valeurs manquantes par la médiane

In [15]:
df_maisons["Surface terrain"] = df_maisons["Surface terrain"].fillna(df_maisons["Surface terrain"].median())

In [16]:
df_maisons.describe()

,Surface reelle bati,Nombre pieces principales,Surface terrain,Nombre de lots,Valeur fonciere,prix_m2
count,354.000000,354.0,354.000000,354.000000,354.000000,354.000000
mean,84.310734,4.0,142.019774,0.039548,251205.864379,3003.594635
std,19.462888,0.0,83.157533,0.195171,105865.398518,1116.900260
min,44.000000,4.0,31.000000,0.000000,16000.000000,144.144144
25%,71.000000,4.0,87.250000,0.000000,185032.500000,2291.390977
50%,80.500000,4.0,127.000000,0.000000,237550.000000,2847.948718
75%,95.000000,4.0,172.750000,0.000000,290000.000000,3481.995492
max,165.000000,4.0,607.000000,1.000000,753750.000000,9192.073171


In [17]:
df_appartements = df_appartements.drop(columns=["Surface terrain"])

In [18]:
df_appartements.describe()

,Surface reelle bati,Nombre pieces principales,Nombre de lots,Valeur fonciere,prix_m2
count,435.000000,435.0,435.000000,4.350000e+02,435.000000
mean,90.450575,4.0,1.434483,4.516911e+05,4989.066032
std,23.340679,0.0,0.865204,6.368548e+05,7087.899307
min,18.000000,4.0,0.000000,1.000000e+00,0.011494
25%,77.000000,4.0,1.000000,2.150000e+05,2617.078231
50%,85.000000,4.0,1.000000,2.850000e+05,3285.714286
75%,99.000000,4.0,2.000000,4.297500e+05,4310.382231
max,215.000000,4.0,8.000000,4.738000e+06,52065.934066
